In [ ]:
import os
import zipfile
import requests
from io import BytesIO
from typing import Tuple, List, Dict, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception, ResNet50, InceptionV3
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers, losses

In [ ]:
# Parameters
img_size = (224, 224)
batch_size = 32

In [ ]:
def create_dataframe_from_directory(path: str) -> pd.DataFrame:
    classes = []
    class_paths = []
    for label in os.listdir(path):
        label_dir = os.path.join(path, label)
        if os.path.isdir(label_dir):
            for image in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image)
                class_paths.append(image_path)
                classes.append(label)
    
    return pd.DataFrame({'Class Path': class_paths, 'Class': classes})

In [ ]:
def plot_class_distribution(df: pd.DataFrame) -> None:
    plt.figure(figsize=(15, 7))
    ax = sns.countplot(data=df, y='Class', hue='Class', palette='viridis', dodge=False)
    
    if ax.get_legend() is not None:
        ax.get_legend().remove()
    
    plt.xlabel('')
    plt.ylabel('')
    plt.title('Count of images in each class', fontsize=20)
    
    for container in ax.containers:
        ax.bar_label(container, label_type='edge', padding=10)
    
    plt.show()

In [ ]:
def plot_sample_images(generator, classes, num_samples=16):
    images, labels = next(generator)
    plt.figure(figsize=(20, 20))
    
    for i in range(num_samples):
        plt.subplot(4, 4, i + 1)
        image = images[i] / 255
        plt.imshow(image)
        class_name = classes[np.argmax(labels[i])]
        plt.title(class_name, color='k', fontsize=20)
    
    plt.show()

In [ ]:
def create_image_generators(df_train: pd.DataFrame, 
                            df_valid: pd.DataFrame, 
                            df_test: pd.DataFrame,
                            batch_size: int = 16, 
                            img_size: Tuple[int, int] = (224, 224)) -> Tuple[ImageDataGenerator, ImageDataGenerator, ImageDataGenerator]:
    train_datagen = ImageDataGenerator(
        brightness_range=(0.7, 1.3), 
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=40
    )
    
    valid_test_datagen = ImageDataGenerator()
    train_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        x_col='Class Path',
        y_col='Class',
        batch_size=batch_size,
        target_size=img_size,
        class_mode='categorical'
    )
    
    valid_gen = valid_test_datagen.flow_from_dataframe(
        dataframe=df_valid,
        x_col='Class Path',
        y_col='Class',
        batch_size=batch_size,
        target_size=img_size,
        class_mode='categorical'
    )
    
    test_gen = valid_test_datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='Class Path',
        y_col='Class',
        batch_size=batch_size,
        target_size=img_size,
        shuffle=False,
        class_mode='categorical'
    )
    
    return train_gen, valid_gen, test_gen

In [ ]:
def build_model(base_model_name: str, num_classes: int) -> Model:
    if base_model_name == 'Xception':
        base_model = Xception(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    elif base_model_name == 'InceptionV3':
        base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    else:
        raise ValueError(f"Unknown base model: {base_model_name}")
    
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x) 
    x = Dense(128, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=Adamax(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
def plot_training_history(history: tf.keras.callbacks.History) -> None:
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def plot_confusion_matrix(y_true: pd.Series, y_pred: List[str], class_names: List[str]) -> None:
    cm = confusion_matrix(y_true, y_pred, labels=class_names)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
# Load and prepare data
train_path = './Dataset/train'
test_path = './Dataset/test'

In [ ]:
plot_loss_curves(history)

In [ ]:
train_df = create_dataframe_from_directory(train_path)
test_df = create_dataframe_from_directory(test_path)

In [ ]:
valid_df, test_df = train_test_split(test_df, train_size=0.5, random_state=20)

In [ ]:
plot_class_distribution(train_df)
plot_class_distribution(test_df)

In [ ]:
train_gen, valid_gen, test_gen = create_image_generators(train_df, valid_df, test_df, batch_size=batch_size, img_size=img_size)

In [ ]:
num_classes = len(train_gen.class_indices)

In [ ]:
# Build and compile model
# base_model_ResNet50 = 'ResNet50'  
# base_model_Xception = 'Xception'  
base_model_InceptionV3 = 'InceptionV3'  

model_ResNet50 = build_model(base_model_InceptionV3, num_classes)

In [ ]:
model_ResNet50.summary()

In [ ]:
# Plot sample images
class_dict = train_gen.class_indices
classes = list(class_dict.keys())
plot_sample_images(train_gen, classes)

### Train the model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3)

In [ ]:
history_ResNet50 = model_ResNet50.fit(
    train_gen,
    epochs=5,
    validation_data=valid_gen,
    shuffle=True,
    callbacks=[early_stopping]
)

In [ ]:
# Plot training history
plot_training_history(history_ResNet50)

In [ ]:
# Evaluate the model
test_gen.reset()
y_pred_ResNet50 = model_ResNet50.predict(test_gen)
y_pred_classes_ResNet50 = np.argmax(y_pred_ResNet50, axis=1)
class_names = list(train_gen.class_indices.keys())

In [ ]:
# Convert predictions to class names
y_true = test_df['Class']
y_true_labels = y_true.astype(str)
y_pred_labels_ResNet50 = [class_names[idx] for idx in y_pred_classes_ResNet50]

In [ ]:
# Print classification report and accuracy
print(classification_report(y_true_labels, y_pred_labels_ResNet50))
print(f"Accuracy: {accuracy_score(y_true_labels, y_pred_labels_ResNet50):.4f}")

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(y_true_labels, y_pred_labels_ResNet50, class_names)

## Xception

In [ ]:
base_model = 'Xception'   
model_Xception = build_model(base_model, num_classes)

In [ ]:
model_Xception.summary()

### Train model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3)

In [ ]:
history_Xception = model_Xception.fit(
    train_gen,
    epochs=5,
    validation_data=valid_gen,
    shuffle=True,
    callbacks=[early_stopping]
)

In [ ]:
# Plot training history
plot_training_history(history_Xception)

In [ ]:
# Evaluate the model
test_gen.reset()
y_pred_Xception = model_Xception.predict(test_gen)
y_pred_classes_Xception = np.argmax(y_pred_Xception, axis=1)
class_names = list(train_gen.class_indices.keys())

In [ ]:
# Convert predictions to class names
y_true = test_df['Class']
y_true_labels = y_true.astype(str)
y_pred_labels_Xception = [class_names[idx] for idx in y_pred_classes_Xception]

In [ ]:
# Print classification report and accuracy
print(classification_report(y_true_labels, y_pred_labels_Xception))
print(f"Accuracy: {accuracy_score(y_true_labels, y_pred_labels_Xception):.4f}")

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(y_true_labels, y_pred_labels_Xception, class_names)

## Modelo Alternativo